In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from datasets import load_dataset
import re
import string
from nltk.corpus import stopwords

In [3]:
ds = load_dataset('imdb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
ds['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [5]:
ds['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [6]:
import pandas as pd
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

In [7]:
ds_train.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [8]:
ds_test.head()

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0


In [9]:
from datasets import Dataset, DatasetDict
#assign the splits
train = Dataset.from_pandas(ds_train)
test = Dataset.from_pandas(ds_test)
#reconstruct both datasets into a Dataset Dict object
new_ds = DatasetDict(
    {
        'train': train,
        'test': test
    }
)
#view the resulting dataset dict object
new_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [10]:
ds_train['text']

,text
0,I rented I AM CURIOUS-YELLOW from my video sto...
1,"""I Am Curious: Yellow"" is a risible and preten..."
2,If only to avoid making this type of film in t...
3,This film was probably inspired by Godard's Ma...
4,"Oh, brother...after hearing about this ridicul..."
...,...
24995,A hit at the time but now better categorised a...
24996,I love this movie like no other. Another time ...
24997,This film and it's sequel Barry Mckenzie holds...
24998,'The Adventures Of Barry McKenzie' started lif...


In [11]:
#Removing html tags
def remove_html_tags(text):
  clean = re.compile('<.*?>')
  return re.sub(clean,'',text)

In [12]:
# Adding the text with no html tags column into the dataframe
ds_train['text_no_html_tags'] = ds_train['text'].apply(lambda x: remove_html_tags(x))
ds_test['text_no_html_tags'] = ds_test['text'].apply(lambda x: remove_html_tags(x))

In [13]:
ds_train['text_no_html_tags'][1]

'"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the s

In [14]:
# Removing punctuation
punctuation = string.punctuation
punctuation

def remove_punctuation(text):
  new_text = "".join([word for word in text if word not in punctuation])
  return new_text

In [15]:
# Adding the text with no punctuation as a new column in the dataframe
ds_train['text_no_punctuation'] = ds_train['text_no_html_tags'].apply(lambda x: remove_punctuation(x))
ds_test['text_no_punctuation'] = ds_test['text_no_html_tags'].apply(lambda x: remove_punctuation(x))

In [16]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
# Tokenize words and remove capitalization
def tokenize_regex(text):
  text = text.lower()
  tokens = re.findall(r'\b\w+\b', text)
  return tokens

In [18]:
ds_train['tokenized_text'] = ds_train['text_no_punctuation'].apply(lambda x: tokenize_regex(x))
ds_test['tokenized_text'] = ds_test['text_no_punctuation'].apply(lambda x: tokenize_regex(x))

In [19]:
ds_train.head()

,text,label,text_no_html_tags,text_no_punctuation,tokenized_text
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,I rented I AM CURIOUS-YELLOW from my video sto...,I rented I AM CURIOUSYELLOW from my video stor...,"[i, rented, i, am, curiousyellow, from, my, vi..."
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"""I Am Curious: Yellow"" is a risible and preten...",I Am Curious Yellow is a risible and pretentio...,"[i, am, curious, yellow, is, a, risible, and, ..."
2,If only to avoid making this type of film in t...,0,If only to avoid making this type of film in t...,If only to avoid making this type of film in t...,"[if, only, to, avoid, making, this, type, of, ..."
3,This film was probably inspired by Godard's Ma...,0,This film was probably inspired by Godard's Ma...,This film was probably inspired by Godards Mas...,"[this, film, was, probably, inspired, by, goda..."
4,"Oh, brother...after hearing about this ridicul...",0,"Oh, brother...after hearing about this ridicul...",Oh brotherafter hearing about this ridiculous ...,"[oh, brotherafter, hearing, about, this, ridic..."


In [20]:
# Removing stopwords
stop_words = stopwords.words('english')

def remove_stopwords(text):
  new_text = [word for word in text if word not in stop_words]
  return new_text

In [21]:
#adding a new column with no stopwords
ds_train['text_no_stopwords'] = ds_train['tokenized_text'].apply(lambda x: remove_stopwords(x))
ds_test['text_no_stopwords'] = ds_test['tokenized_text'].apply(lambda x: remove_stopwords(x))


In [22]:
ds_train.head()

,text,label,text_no_html_tags,text_no_punctuation,tokenized_text,text_no_stopwords
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,I rented I AM CURIOUS-YELLOW from my video sto...,I rented I AM CURIOUSYELLOW from my video stor...,"[i, rented, i, am, curiousyellow, from, my, vi...","[rented, curiousyellow, video, store, controve..."
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"""I Am Curious: Yellow"" is a risible and preten...",I Am Curious Yellow is a risible and pretentio...,"[i, am, curious, yellow, is, a, risible, and, ...","[curious, yellow, risible, pretentious, steami..."
2,If only to avoid making this type of film in t...,0,If only to avoid making this type of film in t...,If only to avoid making this type of film in t...,"[if, only, to, avoid, making, this, type, of, ...","[avoid, making, type, film, future, film, inte..."
3,This film was probably inspired by Godard's Ma...,0,This film was probably inspired by Godard's Ma...,This film was probably inspired by Godards Mas...,"[this, film, was, probably, inspired, by, goda...","[film, probably, inspired, godards, masculin, ..."
4,"Oh, brother...after hearing about this ridicul...",0,"Oh, brother...after hearing about this ridicul...",Oh brotherafter hearing about this ridiculous ...,"[oh, brotherafter, hearing, about, this, ridic...","[oh, brotherafter, hearing, ridiculous, film, ..."


In [23]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [24]:
# Lemmatization of text
def lem_text(text,pos):
  lemm = WordNetLemmatizer()
  word_lemm = [lemm.lemmatize(word,pos) for word in text]
  return word_lemm

In [25]:
#adding a new column with lemmatized text
ds_train['lemmatize_text'] = ds_train['text_no_stopwords'].apply(lambda x: lem_text(x,'n'))
ds_test['lemmatize_text'] = ds_test['text_no_stopwords'].apply(lambda x: lem_text(x,'n'))

In [26]:
#lemmatize the verbs
ds_train['lemmatize_text_v'] = ds_train['text_no_stopwords'].apply(lambda x: lem_text(x,'v'))
ds_test['lemmatize_text_v'] = ds_test['text_no_stopwords'].apply(lambda x: lem_text(x,'v'))

In [27]:
print(ds_train['text'][1])
print(ds_train['text_no_html_tags'][1])
print(ds_train['text_no_punctuation'][1])
print(ds_train['tokenized_text'][1])
print(ds_train['text_no_stopwords'][1])
print(ds_train['lemmatize_text'][1])
print(ds_train['lemmatize_text_v'][1])


"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we're treated to the site of Vincent Gallo's throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the same cann

In [28]:
ds_train_test = ds_train['lemmatize_text'][:100]

,lemmatize_text
0,"[rented, curiousyellow, video, store, controve..."
1,"[curious, yellow, risible, pretentious, steami..."
2,"[avoid, making, type, film, future, film, inte..."
3,"[film, probably, inspired, godard, masculin, f..."
4,"[oh, brotherafter, hearing, ridiculous, film, ..."
...,...
95,"[sometime, 1998, saban, acquired, right, produ..."
96,"[biggest, insult, tmnt, ever, fortunantely, of..."
97,"[like, idea, female, turtle, since, 1987, knew..."
98,"[cannot, stay, indifferent, lars, van, trier, ..."
